In [15]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
from ccfj import GetStationPairs
import pandas as pd
import yaml
import geopandas as gp

In [16]:
import sys
sys.path.append(r'../')
from toollib_standard import maplib
from toollib_standard import mathlib
from toollib_standard import filelib
from toollib_standard import stacklib
from toollib_standard import plotlib

In [17]:
with open('a-project_repar.yml', 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
name_project = proj['name']
#name_project = 'project_repartrition/output_repar_02-01/'               # Harmon server
name_project

'project_repartrition/output_repar_02-04/'

In [18]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project:  /shdisk/rem2/Harmon/F-J/San/project_repartrition/output_repar_02-04/


In [19]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [20]:
name_project_based = info_basic['name_project_based']
dir_project_based = os.path.join(dir_project_workspace, name_project_based)
print('dir_project_based: ', dir_project_based)

dir_project_based:  /shdisk/rem2/Harmon/F-J/San/project/output_FJSJ_17-01/


In [21]:
filename = dir_project_based+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic_based = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi_based = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [22]:
dir_stack = dir_project + info_basic['dir_stack']
dir_ds = dir_project + info_basic['dir_ds']
#d_len = info_basic['d_len']
dir_image = dir_project + info_basic['dir_image']

In [23]:
key_subworks = info_basic['key_subworks']
#key_subworks = ['0']
key_subworks

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79']

In [24]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist()
lat_all = stainfo['latitude'].tolist() 
lon_all = stainfo['longitude'].tolist()

In [25]:
stalistname = dir_project+ info_basic['stalistname']
stainfo_all = pd.read_excel(stalistname,sheet_name=None)

In [26]:
faults = np.load('clark_faults.npy', allow_pickle='TRUE').item()

In [27]:
#%%capture
d_len = 31
c_map = 'jet'
v_min = 0.1
v_max = None
info_basic['v_min'] = v_min
info_basic['v_max'] = v_max
info_basic['c_map'] = c_map
#t0 = info_basic['t0']
#v_tag = info_basic['v_tag']
for key_subwork in key_subworks:
    fig,ax = plt.subplots(3,3,figsize= (22,18))
    # settings for subwork
    
    
    #start = info_basic_bi['start'][key_subwork]
    #interval = info_basic_bi['interval'][key_subwork]
    #flag_time = info_basic_bi['flag_time'][key_subwork]
    start = info_basic['start']
    interval = info_basic['interval']
    flag_time = info_basic['flag_time']

    xlim_f = [0,40]
    xlim_T = [-1,1]

    # plot area
    
    stainfo = stainfo_all[key_subwork]
    nsta = len(stainfo.iloc[:,0])
    StationPairs = GetStationPairs(nsta)
    nPairs = int(len(StationPairs)/2)
    stalist = stainfo['Station'].tolist()
    lat = stainfo['latitude'].tolist() 
    lon = stainfo['longitude'].tolist()
    ax[0][0]=plotlib.plot_area(ax[0][0],lon_all,lat_all,lon,lat)
    for i in range(len(faults)):
        ax[0][0].plot(faults['clark'+str(i+1)]['lon'], faults['clark'+str(i+1)]['lat'], 'k')
    

    # plot ncfs and ncfst
    # linear stack
    """
    outname = str(key_subwork)+'_gather_linear.h5'
    ncffile = h5py.File(dir_stack + outname,'r')
    ncfs_sum_linear = ncffile['ncfs'][:]
    r = ncffile['r'][:]
    ncffile.close()
    # timewindow-filtered stack
    outname = str(key_subwork)+'_gather_timewindow.h5'
    ncffile = h5py.File(dir_stack + outname,'r')
    ncfs_sum_remove = ncffile['ncfs'][:]
    ncffile.close()

    f = info_basic_bi['f']
    t = info_basic_bi['t']
    ncfst_linear = mathlib.freq_time(ncfs_sum_linear)
    ncfst_remove = mathlib.freq_time(ncfs_sum_remove)
    
    title1 = "Linear Stack subarea frequency for "+str(key_subwork)
    #xlim_f = [0,15]
    ax[0][1] = plotlib.plot_ncfs(ax[0][1],f,ncfs_sum_linear,r,title1,xlim_f,0)
    ax[0][2] = plotlib.plot_ncfs(ax[0][2],f,ncfs_sum_remove,r,title1,xlim_f,0)
    """
    # plot fj
    ds = h5py.File(dir_ds+'ds_'+str(key_subwork)+'.h5', 'r')
    ds_linear = ds['ds_linear'][0]
    ds_remove = ds['ds_remove'][0]
    c = np.linspace(info_basic['c_min'],info_basic['c_max'],info_basic['c_num'])
    #c = np.linspace(100,2000,1000)
    """
    title0 = "Linear Stack subarea time for "+str(key_subwork)
    ax[1][0] = plotlib.plot_ncfst(ax[1][0],t,ncfst_linear[start::interval],r[start::interval],title0,flag_time,xlim_T,0)
    title0 = "Remove Stack subarea time for "+str(key_subwork)
    ax[1][1] = plotlib.plot_ncfst(ax[1][1],t,ncfst_remove[start::interval],r[start::interval],title0,flag_time,xlim_T,0)
    for i in range(len(r)):
        ax[1][1].plot()
    """


    
    title0 = "Linear stack dispersion curve "+str(d_len)+' days'
    ax[2][0] = plotlib.plot_fj(ax[2][0],ds_linear,title0,f,c,0,v_min=v_min,v_max=v_max,c_map=c_map)
    ax[2][0].set_xlim(xlim_f)

    title0 = "Remove stack dispersion curve "+str(d_len)+' days'
    ax[2][1] = plotlib.plot_fj(ax[2][1],ds_remove,title0,f,c,0,v_min=v_min,v_max=v_max,c_map=c_map)
    ax[2][1].set_xlim(xlim_f)


    plt.tight_layout()
    plt.savefig(dir_image+str(key_subwork)+'_compare_remove.png',dpi=100)
    plt.close()

In [ ]:
with open(dir_project+'Basic_info.yml', 'w', encoding='utf-8') as f:
   yaml.dump(data=info_basic, stream=f, allow_unicode=True)